In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub

path = kagglehub.dataset_download("emirhanai/emotion-prediction-with-semi-supervised-learning")

print("Path to dataset files:", path)


Path to dataset files: /root/.cache/kagglehub/datasets/emirhanai/emotion-prediction-with-semi-supervised-learning/versions/32


# Sentiment Analysis using Q-NLP


### For backwards compatibiliy
> qml.qnn.KerasLayer  

KerasLayer requires a Keras version lower than 3. For instructions on running with Keras 2
But, tensorflow automatically installs Keras 3

Install tf_keras: `pip install tf_keras`  
Change env variables to use legacy keras

In [ ]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

## Required Libraries

The libraries required for for implementation

In [ ]:
!pip uninstall tensorflow keras -y
!pip install tensorflow==2.9.0 keras==2.9.0
!pip install tensorflow-quantum==0.7.2
!pip install pennylane
!pip install tf_keras
!pip install cirq==0.13.1

Found existing installation: tensorflow 2.17.1
Uninstalling tensorflow-2.17.1:
  Successfully uninstalled tensorflow-2.17.1
Found existing installation: keras 2.9.0
Uninstalling keras-2.9.0:
  Successfully uninstalled keras-2.9.0
  Using cached keras-2.9.0-py2.py3-none-any.whl.metadata (1.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 915.4 kB/s eta 0:00:00
Using cached keras-2.9.0-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 32.4 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 24.3.25
    Uni

ERROR: Could not find a version that satisfies the requirement tensorflow-quantum==0.7.2 (from versions: 0.7.3)
ERROR: No matching distribution found for tensorflow-quantum==0.7.2
  Using cached keras-3.7.0-py3-none-any.whl.metadata (5.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 751.3 kB/s eta 0:00:00
Using cached keras-3.7.0-py3-none-any.whl (1.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 77.9 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
      Successfully uninstalled tensorboard-data-server-0.6.1
 

# Import Libraries
Importing essential libraries for the project

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from keras.models import Model
from tensorflow.keras import layers

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import re

import pennylane as qml
from pennylane import numpy as p_np

from pennylane.templates.state_preparations import MottonenStatePreparation
from pennylane.templates.layers import StronglyEntanglingLayers
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Import the dataset
Import the dataset and prepare the dataframe

In [ ]:
# load our data
data = pd.read_csv("/tweet_emotions.csv")
#plt.style.use('fivethirtyeight')

FileNotFoundError: [Errno 2] No such file or directory: 'P:\\QuantumML\\QML_project\\archive\\tweet_emotions.csv'

## EDA
Exploratory data analysis of the dataset

In [ ]:
data.head()

Shape of the dataset

In [ ]:
data.shape

Columns

In [ ]:
data.columns

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

No null values

# Data Visualization

In [ ]:
sns.countplot(y=data["sentiment"])

# Data Cleanup and Prepare

The original paper proposed using `RegexpStemmer` and remove expressions using  
```snowball = RegexpStemmer('ing$|s$|e$|able$', min=4)```

We propose a `PorterStemmer` and  `WordNetLemmatizer` based approach.
    
 `PorterStemmer` removes the suffix of the word and creates stem word.  
  `WordNetLemmatizer` leverages WordNet, a lexical database for the English Language to reduce words to base or dictionary forms.

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re

# Initialize stop words, stemmer, and lemmatizer
stop_words = set(stopwords.words('english'))
porter_stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Process the content
data["content"] = data["content"].apply(lambda x: " ".join([
    lemmatizer.lemmatize(porter_stemmer.stem(re.sub(r'[^\w\d]', "", word)))
    for word in x.lower().split() if word not in stop_words
]))

# Tokenize and pad sequences
X = data["content"]
tokenize = Tokenizer()
tokenize.fit_on_texts(X)

max_length = 64
vocab_size = len(tokenize.word_index) + 1
X = pad_sequences(tokenize.texts_to_sequences(X), maxlen=max_length, padding="post")

Stemmed and Lemmatized data

In [ ]:
data

Tokenized and encoded data

In [ ]:
X

# Preparation of the Model  
The model architecture consists of several layers that are sequentially connected to form a neural network.   
Architecture:

**Quantum Layer (qlayer)**: This layer is implemented using the qnode function. It takes in two inputs: inputs and weights. The inputs represent the input data, and the weights represent the learnable parameters of the quantum layer.  
Inside the qnode function:  
>The `AngleEmbedding` template is used to encode the input data onto the quantum state.  
  
>The `StronglyEntanglingLayers` template is used to apply a series of parameterized quantum gates to the quantum state.  
  
>Finally, the expectation values of the `Pauli-Z` operators on qubits 0 and 1 are computed using `qml.expval`.  

The output of the quantum layer is a tuple of these two expectation values.

**Dense Layer (clayer1)**: This is a traditional dense layer implemented using the **Dense** class `from tf.keras.layers`. It takes in the output of the quantum layer as input and applies a linear transformation followed by an activation function. The number of neurons in this layer is 2.

**Dense Layer (clayer2)**: This is another dense layer similar to clayer1.

**Dense Layer (clayer3)**: This is the final dense layer of the model. It takes in the output of clayer2 as input and applies a linear transformation followed by a *softmax* activation function. The number of neurons in this layer is 13, which corresponds to the number of classes in the classification task.

The model is compiled using the stochastic gradient descent (SGD) optimizer with a learning rate of 0.8, and the mean absolute error (MAE) loss function is used for training the model.



In [ ]:
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))

weight_shapes = {"weights": (3, n_qubits, 3)}

qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=2)
clayer1 = tf.keras.layers.Dense(2)
clayer2 = tf.keras.layers.Dense(2)
clayer3 = tf.keras.layers.Dense(13, activation="softmax")
model = tf.keras.models.Sequential([clayer1, qlayer, clayer2, clayer3])
opt = tf.keras.optimizers.SGD(learning_rate=0.8)
model.compile(opt, loss='mae')

# Preparation of Label

In [ ]:
Y = data['sentiment']

In [ ]:
Y.value_counts()

In [ ]:
Y

In [ ]:
from sklearn.preprocessing import LabelEncoder

func = LabelEncoder()
Y = func.fit_transform(Y)
Y

In [ ]:
tf.convert_to_tensor(Y)

In [ ]:
tf.one_hot(Y, depth=1)

In [ ]:
print(f"X shape = {X.shape}")
print(f"Y shape = {Y.shape}")

# Model Fitting

In [ ]:
model.fit(X,Y, epochs=5, batch_size=256, shuffle=True,steps_per_epoch=1,validation_split=0.2)

# Preparation of Prediction Data and Prediction Equation

In [ ]:
a = "The loved the movie, the story was amazing and the acting was great."
tokenize.fit_on_texts(a)

max_length = 64
vocab_size = len(tokenize.word_index) + 1
a = pad_sequences(tokenize.texts_to_sequences(a), maxlen=max_length, padding="post")

In [ ]:
a = tf.constant(a)
prediction = model.predict(a)

In [ ]:
ap = []
for i in range(len(prediction)):
    ap.append(np.argmax(prediction[i]))
np.argmax(ap)

In [ ]:
result = np.argmax(ap)
result = np.array(result).reshape(1)

In [ ]:
func.inverse_transform(result)

In [ ]:
result

In [ ]:
a = input("Enter your statement: ")
print(a)

tokenize.fit_on_texts(a)
max_length = 64
vocab_size = len(tokenize.word_index) + 1
a = pad_sequences(tokenize.texts_to_sequences(a), maxlen=max_length, padding="post")
a = tf.constant(a)

output = []

for i in range(10):
    try:

        prediction = model.predict(a)
        ap = []
        for i in range(len(prediction)):
            ap.append(np.argmax(prediction[i]))
        np.argmax(ap)
        result = np.argmax(ap)
        result = np.array(result).reshape(1)
        output.append(func.inverse_transform(result))
    except:
        print("Invalid input")


output